#1. Setup


In [1]:
# virtual display for later video recording
import os
import neat
import numpy as np
import gymnasium as gym


import numpy as np
import os
from pathlib import Path

from tqdm.auto import tqdm
from utils import read_genome, write_genome, animate_generations, laststate
from genome import QuantizedGenome
from reproduce import SoftmaxReproduction
from datetime import datetime as dt
import multiprocessing as mp

/home/dev/miniconda3/envs/c7643/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def log_header():
    with open("log.csv", 'w') as f:
        f.write(",".join(["Timestamp", "Generation", "Best fitness", "Avg fitness"]) + '\n')

def log(data):
    data.insert(0, str(dt.now()))
    with open("log.csv", 'a') as f:
        f.write(",".join(data) + '\n')

In [3]:
def eval_genome(genome, config):
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    fitnesses = []

    for _ in range(1):  # Run the environment 5 times and take the average fitness
        env = gym.make("BipedalWalker-v3", hardcore=False)
        observation, _ = env.reset()
        fitness = 0.0
        max_steps = 1600

        for i in range(max_steps):
            action = net.activate(observation)
            observation, reward, done, truncated, info = env.step(action)
            fitness += reward
            if done or truncated:
                break

        fitnesses.append(fitness)
        env.close()

    return np.mean(fitnesses)

def eval_genomes(genomes, config, storage="models"):
    global gen
    best_fitness = -1e10
    most_fit = None
    avg_fitness = []
    # Get the number of CPU cores
    num_cores = mp.cpu_count()

    # Create a pool with the number of cores minus one
    pool = mp.Pool(processes=num_cores - 1)
    
    results = [pool.apply_async(eval_genome, args=(genome, config)) for _, genome in genomes]
    pool.close()  # Close the pool to prevent new tasks from being submitted

    # Create a progress bar
    pbar = tqdm(total=len(genomes), desc=f"Generation {gen:3}", unit="genome")

    for genome_id, (_, genome) in enumerate(genomes):
        fitness = results[genome_id].get()  # Retrieve the result from the worker process
        genome.fitness = fitness

        avg_fitness.append(genome.fitness)

        if genome.fitness > best_fitness:
            best_fitness = genome.fitness
            most_fit = genome

        # Update the progress bar
        pbar.set_postfix(best_fitness=f"{best_fitness:.3f}", avg_fitness=f"{np.mean(avg_fitness):.3f}")
        pbar.update(1)

    pbar.close()

    # Wait for all worker processes to finish
    for result in results:
        result.wait()

    os.makedirs(storage, exist_ok=True)
    write_genome(most_fit, os.path.join(storage, f"genome_{gen}.pkl"))
    log([str(gen), str(best_fitness), str(np.mean(avg_fitness))])
    # log(f"Gen: {gen:3} | Best fitness: {best_fitness:.3f} | Avg fitness: {np.mean(avg_fitness):.3f}")
    gen += 1

    pool.join() 

In [4]:
# Load the NEAT configuration
config = neat.Config(
    QuantizedGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    'config-quant-bipedal.txt'
)


In [ ]:
# Create the NEAT population
p = neat.Population(config)

# Run the NEAT algorithm for a smaller number of generations
gen = 0
log_header()
best_genome = p.run(eval_genomes, 150)

Generation 141:  67%|██████▋   | 134/200 [00:07<00:03, 18.45genome/s, avg_fitness=59.508, best_fitness=274.075]